In [ ]:
#x = pd.read_csv('../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/name_mapping.csv')

In [ ]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import cv2
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.model_selection import train_test_split

In [ ]:

rootpath = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
rootlist = []
totlist = []
for filename_root in tqdm(np.sort(os.listdir(rootpath))[:-2]):
    subpath = os.path.join(rootpath, filename_root)
    filelist = []
    for filename in np.sort(os.listdir(subpath)):
        filelist.append(os.path.join(subpath, filename))
    rootlist.append(filename_root)
    totlist.append(filelist)
    
maps = pd.concat([pd.DataFrame(rootlist, columns = ['DIR']), 
                  pd.DataFrame(totlist, columns = ['flair', 'seg', 't1', 't1ce', 't2'])], axis = 1)

maps

In [ ]:
name = pd.read_csv('../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/name_mapping.csv')
survive = pd.read_csv('../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/survival_info.csv')

In [ ]:
def DataReader(filepath):
    input = nib.load(filepath)
    return(np.rot90(np.array(input.get_fdata())))

In [ ]:
ctscan_image = DataReader(maps['flair'][0])
fig = plt.figure(figsize = (18, 15))
plt.subplot(2, 2, 1)
plt.imshow(ctscan_image[:, :, 100], cmap = 'bone')
plt.title('Original Image', fontsize = 22.5)

In [ ]:
ctscan_image = DataReader(maps['seg'][0])
fig = plt.figure(figsize = (18, 15))
plt.subplot(2, 2, 1)
plt.imshow(ctscan_image[:, :, 100], cmap = 'bone')
plt.title('Original Image', fontsize = 22.5)

In [ ]:
ctscan_image = DataReader(maps['t1'][0])
fig = plt.figure(figsize = (18, 15))
plt.subplot(2, 2, 1)
plt.imshow(ctscan_image[:, :, 100], cmap = 'bone')
plt.title('Original Image', fontsize = 22.5)

In [ ]:
ctscan_image = DataReader(maps['t1ce'][0])
fig = plt.figure(figsize = (18, 15))
plt.subplot(2, 2, 1)
plt.imshow(ctscan_image[:, :, 100], cmap = 'bone')
plt.title('Original Image', fontsize = 22.5)

In [ ]:
ctscan_image = DataReader(maps['t2'][0])
fig = plt.figure(figsize = (18, 15))
plt.subplot(2, 2, 1)
plt.imshow(ctscan_image[:, :, 100], cmap = 'bone')
plt.title('Original Image', fontsize = 22.5)

In [ ]:
maps

In [ ]:
flair_sampler = DataReader(maps.loc[0,'flair'])
seg_sampler = DataReader(maps.loc[0, 'seg'])
t1_sampler = DataReader(maps.loc[0, 't1'])
t1ce_sampler = DataReader(maps.loc[0, 't1ce'])
t2_sampler = DataReader(maps.loc[0, 't2'])

In [ ]:
SAMPLE = 130

In [ ]:
fig = plt.figure(figsize = (18, 15))
plt.subplot(1, 5, 1)
plt.imshow(flair_sampler[SAMPLE], cmap = 'bone')
plt.title('Original Image')

plt.subplot(1, 5, 2)
plt.imshow(t1ce_sampler[SAMPLE], cmap = 'bone')
plt.imshow(seg_sampler[SAMPLE], alpha = 0.5, cmap = 'nipy_spectral')
plt.title('segment')

plt.subplot(1, 5, 3)
plt.imshow(t1ce_sampler[SAMPLE], cmap = 'bone')
plt.imshow(t1_sampler[SAMPLE], alpha = 0.5, cmap = 'nipy_spectral')
plt.title('t1')

plt.subplot(1, 5, 4)
plt.imshow(t1ce_sampler[SAMPLE], cmap = 'bone')
plt.imshow(t1ce_sampler[SAMPLE], alpha = 0.5, cmap = 'nipy_spectral')
plt.title('t1ce')

plt.subplot(1, 5, 5)
plt.imshow(t1ce_sampler[SAMPLE], cmap = 'bone')
plt.imshow(t2_sampler[SAMPLE], alpha = 0.5, cmap = 'nipy_spectral')
plt.title('t2')

In [ ]:
fig = plt.figure(figsize = (18, 15))
plt.subplot(1, 5, 1)
plt.imshow(flair_sampler[SAMPLE], cmap = 'bone')
plt.title('Original Image')

plt.subplot(1, 5, 2)
plt.imshow(seg_sampler[SAMPLE], cmap = 'bone')
plt.title('brain Mask')

plt.subplot(1, 5, 3)
plt.imshow(t1_sampler[SAMPLE], cmap = 'bone')
plt.title('t1 Mask')

plt.subplot(1, 5, 4)
plt.imshow(t1ce_sampler[SAMPLE], cmap = 'bone')
plt.title('t1ce Mask')

plt.subplot(1, 5, 5)
plt.imshow(t2_sampler[SAMPLE], cmap = 'bone')
plt.title('t2')

In [ ]:
SAMPLE = 41

fig = plt.figure(figsize = (10, 5))
plt.subplot(1, 2, 1)
plt.imshow(flair_sampler[SAMPLE], cmap = 'bone')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(seg_sampler[SAMPLE], cmap = 'bone')
plt.title('brain Mask')

plt.tight_layout()

In [ ]:
brains = []
segs = []
img_size = 128

for i in tqdm(range(len(maps))):
    ct = DataReader(maps['flair'][i])
    infect = DataReader(maps['seg'][i])
    for ii in range(ct.shape[0]):
        brain_img = cv2.resize(ct[ii], dsize = (img_size, img_size),interpolation = cv2.INTER_AREA).astype('uint8')
        infec_img = cv2.resize(infect[ii],dsize=(img_size, img_size),interpolation = cv2.INTER_AREA).astype('uint8')
        brains.append(brain_img[..., np.newaxis])
        segs.append(infec_img[..., np.newaxis])

In [ ]:
brains = np.array(brains)
segs = np.array(segs)

print(brains.shape)
print(segs.shape)

In [ ]:
plt.imshow(brains[6090])

In [ ]:
plt.imshow(segs[6090])

In [ ]:
brain_train, brain_test, infect_train, infect_test = train_test_split(brains, segs, test_size = 0.5)

In [ ]:
def build_model(input_layer, start_neurons):
    # 128 -> 64
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    # 64 -> 32
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    # 32 -> 16
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    # 16 -> 8
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)

    # 8 -> 16
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    # 16 -> 32
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    # 32 -> 64
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    # 64 -> 128
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)

    #uconv1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

input_layer = Input((img_size, img_size, 1))
output_layer = build_model(input_layer, 16)

model = Model(input_layer, output_layer)
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:

plot_model(model, 
           show_shapes = True, 
           show_layer_names = True, 
           rankdir = 'TB', 
           expand_nested = False, 
           dpi = 60)

In [ ]:
history = model.fit(brain_train, infect_train, epochs = 10, validation_data = (brain_test, infect_test))

In [ ]:
import gc
gc.collect()

In [ ]:
predicted = model.predict(brain_test[:100])


In [ ]:
sample = 4

In [ ]:
fig = plt.figure(figsize = (18, 15))
plt.subplot(1,3,1)
plt.imshow(brain_test[sample])
plt.title('original')

plt.subplot(1,3,2)
plt.imshow(predicted[sample])
plt.title('predicted infection mask')